# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import pickle

In [166]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [6]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [ ]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[2657:3041]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

In [162]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,9791129705297,[9.2],"[72%, 21%, 8%, 0%, 0%]","[모카우유, 사랑해, 서평]모카우유, 사랑해, '모카우유, 사랑해' 랜선 형님이 되...","[져*, 깡***나, 장*님, 아**3, 코**스, n*******g, x*****...","[5점, 5점, 5점, 3점, 5점, 5점, 5점, 5점, 5점, 5점]","[\n 여기는 모카,우유네 입니다.~^♥ㅋ딱 봐도 누가 모카고, 누가 우유인지 한눈..."
1,9791190920193,[9.9],"[100%, 0%, 0%, 0%, 0%]","[어이쿠 오늘도 행복했네, 어이쿠, 오늘도 행복했네/b.read(브레드), 브레드 ...","[u****e, 책**맘, 눈*신, 김*철, o*****8, 책****다, 혜*, ...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]","[\n어이쿠, 오늘도 행복했네 - 이정수 \n \n \n \n썰렁해지면 돌아온..."
2,9788967820602,[9.5],"[82%, 18%, 0%, 0%, 0%]","[『아름다운 배웅』 '웰다잉' 아름다운 죽음을 원하는 이들에게 권한다, 아름다운 배...","[h****n, n*****m, 여*다, k****5, 은****기, p******...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n지은이: 심은이펴낸이: 한효정펴낸곳: 도서출판 푸른향기 국내 첫 여성 장례지도...
3,9788993635065,[9.4],"[67%, 33%, 0%, 0%, 0%]",[당신의 관계를 객관적으로 파악하는데 도움이 되는 책┃그 남자는 절대 변하지 않는다...,"[나*벨, C*****C, 북*, 윤**빠, q******7, a********* ...","[4점, 4점, 5점, 5점, 5점, 5점, 5점, 5점, 4점]",[\n 학대가 나쁘다는 건 다 안다. 만일 누군가 당신에게 학대를 당하고 있는 사람...
4,9788954643146,[8.5],"[43%, 57%, 0%, 0%, 0%]","[세월의 더께, [2016 결산] 건너가자, 건너오다 - 낯섦이 주는 위안, 김현우...","[꼼*, 싱*, E**y, H***M, 건****탕, M******d, k******i]","[5점, 5점, 4점, 5점, 4점, 5점, 3점]",[\n어떤 장면은 세월에 따라 세세한 풍경은 잊히지만 공간 깊숙이 스며들었던 소리는...
5,9791185823072,[9.2],"[57%, 43%, 0%, 0%, 0%]","[영국 코미디언 이안 무어의 문화충돌기『영국에서 사흘 프랑스에서 나흘』, 영국에서 ...","[금*, 껌***스, 책****벤, j****9, a******m, 북*더, c**...","[4점, 5점, 5점, 5점, 5점, 4점, 4점]",[\n첫 장부터 끝장까지 킬킬대며 읽을 수 있는 책이다. '이안 무어'라는 영국 코...
6,9791160100150,[],[],[],[],[],[]
7,9791197560217,[10.0],"[100%, 0%, 0%, 0%, 0%]","[어쩌다 백화점, 작가를 꿈꾸는 여자의 백화점 근무일지 [어쩌다 백화점]]","[k*******9, 뿌*]","[5점, 5점]",[\n\n소리없는 전쟁터!! 이곳은 백화점이야기와 작가님의 생각과 철학있는 책이야기...
8,9788954624923,[8.8],"[55%, 45%, 0%, 0%, 0%]","[어떤 지독한 기억과의 해후, '용산'이라는 거대한 이름의 시詩 [지나치게 산문적인...","[자*련, 이*새, w*******i, 안*슝, 오***객, s*****0, 보*숲...","[4점, 5점, 5점, 4점, 5점, 4점, 4점, 4점, 4점, 4점]","[\n\n \n \n ‘멀리서 보면 장소는 무심하고 자명하며, 가까이서 보면 장소는..."
9,9788974188221,[9.2],"[72%, 22%, 6%, 0%, 0%]","[실천을 담보로 한 선택만이 삶을 바꾼다., 7년의 기록, 남자 간호사 데이비드 이...","[초*, 책*사, 산*람, i*****n, 신*****리, 이*, A***n, 가*...","[4점, 4점, 5점, 3점, 5점, 4점, 5점, 5점, 5점, 5점]",[\n\n자기계발서에 대해서는 양가적인 감정을 갖고 있다. 모든 것이 다 너의 잘못...


In [164]:
for i in range(2597,3041):
    if isbn13[i] == '9788961557566':
        print(i)

2656


In [165]:
user.to_pickle('user_2597-2656.pkl', protocol = 4)

In [120]:
user1 = pd.read_pickle('user_1700-1986..pkl') 